In [1]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(8,3.5))

<Figure size 800x350 with 0 Axes>

In [2]:
df1 = pd.read_csv('2018年氮肥施用量(农田汇总).csv')
df1 = df1.fillna(0)

In [3]:
df2 = pd.read_csv('2018年氮肥施用量(汇总).csv')
df2 = df2.fillna(0)

In [4]:
file_path = '/Users/xihuan/Desktop/土壤信息数据集/4.数据整理/prediction/EF_prediction/'
file_names = sorted([file for file in os.listdir(file_path) if file.endswith('.csv')], key=lambda x: len(x))
file_names

['EF_predicted(clear_datas3).csv',
 'EF_predicted(clear_datas234).csv',
 'EF_predicted(clear_datas124).csv',
 'EF_predicted(clear_datas123).csv',
 'EF_predicted(clear_datas134).csv',
 'EF_predicted(clear_datas1234).csv',
 'EF_predicted(clear_datas3(9.2)).csv',
 'EF_predicted(Supplementary(6.6晚)).csv',
 'EF_predicted(Supplementary(6.8晚)).csv',
 'EF_predicted(Supplementary(6.2晚)).csv']

In [5]:
dataset_names = []
nh3_prediction = []
nh3_std = []
all_data = dict()

for filename in file_names:

    df_ef =  pd.read_csv(file_path+filename)
    df_ef = df_ef.merge(pd.DataFrame({'grid_num':range(1, 360*720+1)}), on='grid_num', how='right')
    df_ef = df_ef.fillna(0)

    #水稻、玉米和小麦三种作物各种两季，需将网格对应的EF值乘以6来算一年的NH3排放量
    # print(f'NH3农田排放({'EF_predicted(clear_datas1234).csv'.rsplit('.',1)[0].rsplit('(',1)[1].rsplit(')',1)[0]})：{6 * sum(df1['fertilizer_input'].values * df_ef['EF_predicted'].values)/100*1e-12} Tg N/yr')
    title = f'NH3排放({filename.rsplit(".",1)[0].split("(",1)[1].rsplit(")",1)[0]})'
    value = format(6 * sum(df2['fertilizer_input'].values * df_ef['EF_predicted'].values)/100, '.2e')

    title_std = f'样本标准差({filename.rsplit(".",1)[0].split("(",1)[1].rsplit(")",1)[0]})'
    sample_std = format(6 * sum(df2['fertilizer_input'].values * df_ef['std_predicted'].values)/100, '.2e')
    
    dataset_names.append(filename.rsplit(".",1)[0].split("(",1)[1].rsplit(")",1)[0])
    nh3_prediction.append(value)
    nh3_std.append(sample_std)
    all_data[f'{filename.rsplit(".",1)[0].split("(",1)[1].rsplit(")",1)[0]}'] = df_ef['EF_predicted'].values

result = pd.DataFrame({'Dataset': dataset_names, 'NH3 emission prediction（g）': nh3_prediction, 'Samples std（g）': nh3_std})
print(result)

               Dataset NH3 emission prediction（g） Samples std（g）
0         clear_datas3                   3.35e+12       2.68e+10
1       clear_datas234                   3.00e+12       2.46e+10
2       clear_datas124                   4.01e+12       3.12e+10
3       clear_datas123                   3.06e+12       2.50e+10
4       clear_datas134                   3.13e+12       2.54e+10
5      clear_datas1234                   2.91e+12       2.38e+10
6    clear_datas3(9.2)                   3.20e+12       2.59e+10
7  Supplementary(6.6晚)                   3.80e+12       2.97e+10
8  Supplementary(6.8晚)                   3.89e+12       3.05e+10
9  Supplementary(6.2晚)                   4.00e+12       3.13e+10
